In [ ]:
# Install / upgrade core dependencies (safe to re-run). Comment out versions to use latest.
%pip install \
  transformers==4.44.2 \
  datasets==2.21.0 \
  evaluate==0.4.2 \
  accelerate==0.34.2 \
  sentencepiece==0.2.0 \
  nltk==3.9.1 \
  rouge-score==0.1.2 \
  bert-score==0.3.13 \
  sacrebleu==2.4.3 \
  scikit-learn==1.5.2 \
  pandas==2.2.2 \
  numpy==1.26.4 \
  torch --upgrade --quiet

import nltk
nltk.download('punkt', quiet=True)
print('Dependencies installed and punkt tokenizer downloaded.')

### Environment & Dependencies

Run the next cell once per environment to ensure all required packages (with tested versions) are installed before training/fine-tuning.

Core libraries:
- transformers (seq2seq model + trainer)
- datasets (dataset handling)
- evaluate (metrics hub)
- accelerate (efficient distributed / mixed precision)
- sentencepiece (tokenization backend for T5)
- nltk (optional: sentence tokenization, metrics)
- rouge_score, bert_score, sacrebleu (evaluation)
- scikit-learn (train/validation split)
- numpy, pandas (general utilities)

Pin versions if you need reproducibility; here we choose recent stable versions that work well with T5-base. Adjust as needed for CUDA / platform constraints.

### Continued Fine-Tuning: Combined Extended + Schema Datasets

This section loads the two newly generated datasets:
- `bldg3_dataset_extended.json` (semantic sensor + analytic queries)
- `Transformers/t5_base/training/bldg3/bldg3_schema_dataset.json` (ontology/TBox + structural queries)

It merges them into a single training corpus, converts multi-entity lists into a flat string for the model input, and continues fine-tuning from the latest prior checkpoint (if available) or the base `t5-base` model. New checkpoints will be written under `./trained/combined_t5/`.

You can adjust hyperparameters (epochs, batch size, lr) in the following cells if needed.

In [4]:
# Install / upgrade core dependencies (safe to re-run). Comment out versions to use latest.
%pip install \
  transformers==4.44.2 \
  datasets==2.21.0 \
  evaluate==0.4.2 \
  accelerate==0.34.2 \
  sentencepiece==0.2.0 \
  nltk==3.9.1 \
  rouge-score==0.1.2 \
  bert-score==0.3.13 \
  sacrebleu==2.4.3 \
  scikit-learn==1.5.2 \
  pandas==2.2.2 \
  numpy==1.26.4 \
  torch --upgrade --quiet

import nltk
nltk.download('punkt', quiet=True)
print('Dependencies installed and punkt tokenizer downloaded.')

Note: you may need to restart the kernel to use updated packages.
Dependencies installed and punkt tokenizer downloaded.
Dependencies installed and punkt tokenizer downloaded.


In [1]:
!pip install transformers datasets evaluate rouge-score sacrebleu nltk bert-score scikit-learn


In [3]:
# !pip install torch --index-url https://download.pytorch.org/whl/cu118

In [2]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
Transformers/t5_base/T5-base.ipynb

In [ ]:
# Quick smoke test on a couple of random samples after training (run after previous cell finishes)
import random, json, torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from pathlib import Path

model_dir = './trained/checkpoint-3/final'
if not Path(model_dir).exists():
    print('Final model directory not found, please run the training cell first.')
else:
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    model = T5ForConditionalGeneration.from_pretrained(model_dir)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    with open('merged_combined_corpus.json','r',encoding='utf-8') as f:
        merged = json.load(f)

    samples = random.sample(merged, k=min(3, len(merged)))
    for i,s in enumerate(samples,1):
        inp = s['input_text']
        tgt = s['target_text']
        inputs = tokenizer(inp, return_tensors='pt', truncation=True, padding='max_length', max_length=512).to(device)
        out_ids = model.generate(inputs['input_ids'], max_length=256, num_beams=4)
        pred = tokenizer.decode(out_ids[0], skip_special_tokens=True)
        print(f'--- Sample {i} ---')
        print('INPUT:\n', inp[:400])
        print('TARGET:\n', tgt)
        print('PRED:\n', pred)
        print('-'*60)

In [ ]:
# Patch: Safe decoding & debug utilities to prevent IndexError during evaluation
import torch, json, math, statistics
from typing import List

# 1. Verify tokenizer/model alignment again
try:
    embedding_size = model.get_input_embeddings().weight.shape[0]
    vocab_size = len(tokenizer)
    print(f"[DEBUG] Embedding size: {embedding_size} | Tokenizer vocab: {vocab_size}")
    if embedding_size != vocab_size:
        print("[WARNING] Mismatch detected. This can cause out-of-range token ids during decode.")
        print("Suggested fix: Reload both model and tokenizer from the SAME checkpoint or resize embeddings.")
except Exception as e:
    print(f"[DEBUG] Could not inspect model/tokenizer sizes: {e}")

# 2. Utility to clamp OOB ids BEFORE batch_decode

def clamp_oob_ids(seqs: List[List[int]], vocab_limit: int) -> int:
    """Clamp any token id >= vocab_limit or < 0 to pad_token id; returns count of corrections."""
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
    fixes = 0
    for s in seqs:
        for i, tid in enumerate(s):
            if tid >= vocab_limit or tid < 0:
                s[i] = pad_id
                fixes += 1
    return fixes

# 3. Wrap original compute_metrics if not already wrapped
if 'original_compute_metrics' not in globals():
    original_compute_metrics = compute_metrics  # preserve

    def safe_compute_metrics(eval_pred):
        preds, labels = eval_pred
        # preds may be a tuple when using generate with return dict
        if isinstance(preds, tuple):
            preds = preds[0]
        preds_list = preds.tolist() if isinstance(preds, torch.Tensor) else preds
        # Clamp out-of-range token ids
        vocab_limit = len(tokenizer)
        fix_count = clamp_oob_ids(preds_list, vocab_limit)
        if fix_count:
            print(f"[SAFE-DECODE] Corrected {fix_count} out-of-range token ids before decoding.")
        # Convert back to tensor for downstream code expecting tensor shape
        preds_tensor = torch.tensor(preds_list, device=preds.device if isinstance(preds, torch.Tensor) else 'cpu')
        return original_compute_metrics((preds_tensor, labels))

    compute_metrics = safe_compute_metrics
    trainer.compute_metrics = compute_metrics
    print("[PATCH] Installed safe_compute_metrics wrapper to prevent IndexError.")
else:
    print("[PATCH] safe_compute_metrics already installed; skipping re-wrap.")

# 4. Optional immediate evaluation test (uncomment to dry-run metrics without full training)
# try:
#     print("[DRY-RUN] Starting quick evaluation to test decoding safety...")
#     trainer.evaluate(max_length=64)
#     print("[DRY-RUN] Evaluation successful.")
# except Exception as e:
#     print("[DRY-RUN] Evaluation failed:", e)

print("Patch cell executed. You can now re-run training: trainer.train() (ideally from the cell above).")

### Metrics Fallback Explanation
If `evaluate.load('rouge')` fails (e.g., offline environment, hub connectivity, or cache corruption), the previous cell now:

- Tries to `pip install` a required package (e.g. `rouge-score`, `sacrebleu`, `nltk`, `bert-score`).
- If that still fails, it uses a lightweight local fallback implementation for: ROUGE (rouge-score), BLEU (sacrebleu), METEOR (nltk), BERTScore (bert-score).
- Prints which metrics are using fallbacks.

Usage Notes:
- To force re-download from Hugging Face Hub later, ensure network is available and re-run the cell.
- If you want to skip a metric entirely, you can comment out its block in `compute_metrics`.
- Starting training: uncomment `trainer.train()` at the bottom of the previous cell.

Troubleshooting:
- If BERTScore is slow on CPU, you can remove it or set a smaller batch by modifying the fallback.
- For deterministic runs, set `seed` in `Seq2SeqTrainingArguments` and also `random.seed`, `numpy.random.seed`, `torch.manual_seed` before training.


In [ ]:
# === Unified Single-Cell Training Pipeline (Extended + Schema, Multi-Building) ===
# This cell can be run standalone after opening the notebook. It will:
#  1. Ensure core dependencies are present.
#  2. Load merged extended + schema datasets (with provenance).
#  3. Normalize and assemble NL->SPARQL training pairs.
#  4. Train/validation split.
#  5. Load tokenizer & model from latest checkpoint (fallback to t5-base).
#  6. Install robust metric fallback system (ROUGE, BLEU, METEOR, BERTScore).
#  7. Patch compute_metrics with safe decode guard against out-of-range ids.
#  8. Train with early stopping & automatic best-model selection.
#  9. Save final model + tokenizer.
# 10. Run a quick post-training generation smoke test.
# -----------------------------------------------------------------------------

import os, sys, json, random, gc, math, statistics, traceback, subprocess
from pathlib import Path
from typing import List

# ---------------- Configuration ----------------
SEED = 123
EXTENDED_PATH = Path('./training/merged_extended_datasets.json')
SCHEMA_PATH   = Path('./training/merged_schema_datasets.json')
OUTPUT_BASE   = Path('./trained/unified_run')  # new consolidated output dir
PREV_CHECKPOINT_CANDIDATES = [
    './trained/checkpoint-2',
]
BASE_MODEL_NAME = 't5-base'
TRAIN_VAL_SPLIT = 0.05
TARGET_EPOCHS   = 5
LEARNING_RATE   = 3e-4
EVAL_STRATEGY   = 'epoch'
BATCH_PER_DEVICE = 4
GRAD_ACC_STEPS   = 4
MAX_SOURCE_LEN = 512
MAX_TARGET_LEN = 256
EARLY_STOP_PATIENCE = 3
METRIC_FOR_BEST = 'rougeL'
SAVE_TOTAL_LIMIT = 3
GEN_MAX_LEN = 512
GEN_NUM_BEAMS = 1  # greedy-like (beam=1) for stability
USE_FP16 = True  # auto-disabled if no CUDA

# ---------------- Reproducibility ----------------
random.seed(SEED)
try:
    import numpy as np
    np.random.seed(SEED)
except Exception:
    pass
import torch
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# ---------------- Dependency Check (minimal) ----------------
REQUIRED = [
    ('transformers', 'transformers'),
    ('datasets', 'datasets'),
    ('evaluate', 'evaluate'),
    ('sentencepiece', 'sentencepiece'),
    ('nltk', 'nltk'),
    ('sacrebleu', 'sacrebleu'),
    ('rouge_score', 'rouge-score'),
    ('bert_score', 'bert-score'),
    ('scikit_learn', 'scikit-learn'),
]
for import_name, pip_name in REQUIRED:
    try:
        __import__(import_name)
    except ImportError:
        print(f"[Deps] Installing missing package: {pip_name}")
        subprocess.run([sys.executable, '-m', 'pip', 'install', '--quiet', pip_name], check=False)

import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt', quiet=True)

# ---------------- Dataset Loading ----------------
assert EXTENDED_PATH.exists(), f"Missing extended dataset at {EXTENDED_PATH}"
assert SCHEMA_PATH.exists(),   f"Missing schema dataset at {SCHEMA_PATH}"

with EXTENDED_PATH.open('r', encoding='utf-8') as f:
    extended_data = json.load(f)
with SCHEMA_PATH.open('r', encoding='utf-8') as f:
    schema_data = json.load(f)
print(f"[Load] Extended: {len(extended_data)} | Schema: {len(schema_data)}")

# ---------------- Normalization ----------------
def normalize(entry):
    q = (entry.get('question') or '').strip()
    sparql = (entry.get('sparql') or '').strip()
    entities = entry.get('entities') or entry.get('entity') or []
    if isinstance(entities, str):
        entities_list = [e.strip() for e in entities.split('\n') if e.strip()]
    else:
        entities_list = entities
    entity_block = '\n'.join(entities_list) if entities_list else ''
    return {
        'question': q,
        'entities_list': entities_list,
        'entity_block': entity_block,
        'sparql': sparql
    }

normalized = [normalize(e) for e in extended_data] + [normalize(e) for e in schema_data]
print(f"[Normalize] Total normalized: {len(normalized)}")

inputs, targets = [], []
for rec in normalized:
    if rec['question'] and rec['sparql']:
        ent_part = f"\nentity: {rec['entity_block']}" if rec['entity_block'] else ''
        inputs.append(f"task: generate_sparql\ninput: {rec['question']}{ent_part}")
        targets.append(rec['sparql'])
print(f"[Pairs] Valid NL->SPARQL pairs: {len(inputs)}")

# ---------------- Split ----------------
from sklearn.model_selection import train_test_split
train_inputs, val_inputs, train_targets, val_targets = train_test_split(
    inputs, targets, test_size=TRAIN_VAL_SPLIT, random_state=SEED
)
print(f"[Split] Train: {len(train_inputs)} | Val: {len(val_inputs)}")

# Save combined corpus (optional)
with open('merged_combined_corpus.json', 'w', encoding='utf-8') as f:
    json.dump([{'input_text': i, 'target_text': t} for i, t in zip(inputs, targets)], f, ensure_ascii=False, indent=2)
print('[Save] merged_combined_corpus.json written')

# ---------------- Model / Tokenizer Source ----------------
model_source = None
for cand in PREV_CHECKPOINT_CANDIDATES:
    if Path(cand).exists():
        model_source = cand
        break
if model_source is None:
    model_source = BASE_MODEL_NAME
print(f"[Model] Using source: {model_source}")

from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq, EarlyStoppingCallback, TrainerCallback
)

# Tokenizer
try:
    tokenizer = T5Tokenizer.from_pretrained(model_source)
    print(f"[Tokenizer] Loaded from {model_source} | vocab={len(tokenizer)}")
except Exception as e:
    print(f"[Tokenizer] Failed from {model_source}: {e} -> fallback to {BASE_MODEL_NAME}")
    tokenizer = T5Tokenizer.from_pretrained(BASE_MODEL_NAME)

# Model
try:
    model = T5ForConditionalGeneration.from_pretrained(model_source, device_map=None)
    print(f"[Model] Loaded: {model_source}")
except Exception as e:
    print(f"[Model] Failed from {model_source}: {e} -> fallback to {BASE_MODEL_NAME}")
    model = T5ForConditionalGeneration.from_pretrained(BASE_MODEL_NAME)

# Verify embedding vs vocab
embed_size = model.get_input_embeddings().weight.shape[0]
if embed_size != len(tokenizer):
    print(f"[WARNING] Vocab mismatch (embed={embed_size} vs tok={len(tokenizer)}). Aborting to prevent runtime errors.")
    raise SystemExit(1)
else:
    print(f"[Check] Vocab sizes aligned: {embed_size}")

# ---------------- Device + Memory ----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"[Device] Using: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Mem: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")
else:
    print("[Warn] No GPU detected; training will be slow.")

model.to(device)

# ---------------- Metrics Fallback System ----------------
import evaluate

def safe_load_metric(name, pip_pkg=None, alt=None):
    if name in ['rouge', 'bleu', 'meteor'] and alt:
        print(f"[Metric] Direct fallback for {name}")
        return alt, True
    try:
        m = evaluate.load(name)
        def _call(preds, refs):
            return m.compute(predictions=preds, references=refs)
        return _call, False
    except Exception as e:
        print(f"[Metric] Failed load {name}: {e}")
        if pip_pkg:
            subprocess.run([sys.executable,'-m','pip','install','--quiet',pip_pkg], check=False)
            try:
                m = evaluate.load(name)
                def _call(preds, refs):
                    return m.compute(predictions=preds, references=refs)
                return _call, False
            except Exception:
                pass
        if alt:
            print(f"[Metric] Using fallback for {name}")
            return alt, True
        return (lambda p,r: {}), True

# Fallbacks
from rouge_score import rouge_scorer

def rouge_fallback(preds, refs):
    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
    r1=r2=rl=0; n=len(preds)
    for p,r in zip(preds, refs):
        s=scorer.score(r,p)
        r1+=s['rouge1'].fmeasure; r2+=s['rouge2'].fmeasure; rl+=s['rougeL'].fmeasure
    return {'rouge1': r1/n if n else 0.0,'rouge2': r2/n if n else 0.0,'rougeL': rl/n if n else 0.0}

import sacrebleu

def bleu_fallback(preds, refs):
    return {'bleu': sacrebleu.corpus_bleu(preds,[refs]).score}

from nltk.translate.meteor_score import meteor_score

def meteor_fallback(preds, refs):
    scores=[meteor_score([r],p) for p,r in zip(preds,refs)]
    return {'meteor': sum(scores)/len(scores) if scores else 0.0}

try:
    from bert_score import score as bert_score
    def bertscore_fallback(preds, refs):
        P,R,F = bert_score(preds, refs, lang='en', verbose=False)
        return {'bertscore_precision': float(P.mean()), 'bertscore_recall': float(R.mean()), 'bertscore_f1': float(F.mean())}
except Exception:
    def bertscore_fallback(preds, refs):
        return {'bertscore_f1': 0.0}

metric_rouge, r_fb = safe_load_metric('rouge', 'rouge-score', rouge_fallback)
metric_bleu,  b_fb = safe_load_metric('bleu', 'sacrebleu', bleu_fallback)
metric_meteor,m_fb = safe_load_metric('meteor','nltk', meteor_fallback)
metric_bertscore, bs_fb = safe_load_metric('bertscore','bert-score', bertscore_fallback)
print('[Metrics] Fallback usage:', {'rouge':r_fb,'bleu':b_fb,'meteor':m_fb,'bertscore':bs_fb})

# ---------------- Dataset Objects ----------------
from datasets import Dataset, DatasetDict
raw_datasets = DatasetDict({
    'train': Dataset.from_dict({'input_text': train_inputs, 'target_text': train_targets}),
    'validation': Dataset.from_dict({'input_text': val_inputs, 'target_text': val_targets}),
})

label_pad_token_id = -100

def preprocess(batch):
    model_inputs = tokenizer(batch['input_text'], max_length=MAX_SOURCE_LEN, truncation=True, padding='max_length')
    labels = tokenizer(text_target=batch['target_text'], max_length=MAX_TARGET_LEN, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized = raw_datasets.map(preprocess, batched=True, remove_columns=['input_text','target_text'])

from transformers import DataCollatorForSeq2Seq
collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=label_pad_token_id)

# ---------------- Metrics Callback ----------------
from transformers import TrainerCallback
class MetricsTableCallback(TrainerCallback):
    def __init__(self, save_csv_path='epoch_metrics_log.csv'): self.save_csv_path=save_csv_path; self.rows=[]
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        row={'step': state.global_step}
        for k,v in metrics.items():
            if isinstance(v,(int,float)): row[k]=v
        self.rows.append(row); return control
    def on_train_end(self, args, state, control, **kwargs):
        if not self.rows: return
        import csv
        keys=sorted({k for r in self.rows for k in r.keys()})
        with open(self.save_csv_path,'w',newline='',encoding='utf-8') as f:
            w=csv.DictWriter(f, fieldnames=keys); w.writeheader(); [w.writerow(r) for r in self.rows]
        print(f"[Metrics] Saved log -> {self.save_csv_path}")
metrics_callback = MetricsTableCallback()

# ---------------- Compute Metrics (wrapped safe decode) ----------------

def base_compute(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple): preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels= [l.strip() for l in decoded_labels]
    rouge_res = metric_rouge(decoded_preds, decoded_labels)
    bleu_res  = metric_bleu(decoded_preds, decoded_labels)
    meteor_res= metric_meteor(decoded_preds, decoded_labels)
    bert_res  = metric_bertscore(decoded_preds, decoded_labels)
    metrics = {}
    for d in (rouge_res, bleu_res, meteor_res, bert_res): metrics.update({k: float(v) for k,v in d.items()})
    return metrics

# Guard against OOB ids

def clamp_oob_ids(seqs: List[List[int]], vocab_limit: int) -> int:
    pad_id = tokenizer.pad_token_id or 0
    fixes=0
    for s in seqs:
        for i,tok in enumerate(s):
            if tok < 0 or tok >= vocab_limit:
                s[i]=pad_id; fixes+=1
    return fixes


def safe_compute(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple): preds = preds[0]
    if isinstance(preds, torch.Tensor):
        seqs = preds.tolist()
        fix_count = clamp_oob_ids(seqs, len(tokenizer))
        if fix_count: print(f"[SafeDecode] Corrected {fix_count} OOB token ids")
        preds = torch.tensor(seqs, device=preds.device)
    return base_compute((preds, labels))

compute_metrics = safe_compute

# ---------------- Training Arguments ----------------
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
OUTPUT_BASE.mkdir(parents=True, exist_ok=True)
train_args = Seq2SeqTrainingArguments(
    output_dir=str(OUTPUT_BASE),
    evaluation_strategy=EVAL_STRATEGY,
    save_strategy=EVAL_STRATEGY,
    logging_strategy='steps',
    logging_steps=250,
    per_device_train_batch_size=BATCH_PER_DEVICE,
    per_device_eval_batch_size=BATCH_PER_DEVICE,
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    num_train_epochs=TARGET_EPOCHS,
    learning_rate=LEARNING_RATE,
    warmup_steps=200,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=(torch.cuda.is_available() and USE_FP16),
    generation_max_length=GEN_MAX_LEN,
    generation_num_beams=GEN_NUM_BEAMS,
    load_best_model_at_end=True,
    metric_for_best_model=METRIC_FOR_BEST,
    greater_is_better=True,
    save_total_limit=SAVE_TOTAL_LIMIT,
    report_to=[],
    seed=SEED,
    dataloader_num_workers=0 if sys.platform.startswith('win') else 2,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=EARLY_STOP_PATIENCE), metrics_callback]
)

print('\n=== TRAIN CONFIG SUMMARY ===')
print(f"Device: {device} | FP16: {train_args.fp16}")
print(f"Train samples: {len(tokenized['train'])} | Val samples: {len(tokenized['validation'])}")
print(f"Epochs: {train_args.num_train_epochs} | LR: {train_args.learning_rate}")
print(f"Effective batch size: {train_args.per_device_train_batch_size * train_args.gradient_accumulation_steps}")
print(f"Output dir: {train_args.output_dir}")
print('============================\n')

train_result = trainer.train()
print('[Train] Finished training.')
metrics = train_result.metrics

# Save best model + tokenizer
trainer.save_model(str(OUTPUT_BASE / 'final'))
tokenizer.save_pretrained(str(OUTPUT_BASE / 'final'))
print(f"[Save] Final model saved to {OUTPUT_BASE / 'final'}")

# Save metrics JSON
overall_metrics_path = OUTPUT_BASE / 'training_metrics.json'
with overall_metrics_path.open('w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=2)
print(f"[Save] Training metrics -> {overall_metrics_path}")

# Quick smoke test
try:
    from transformers import T5ForConditionalGeneration, T5Tokenizer
    smoke_tokenizer = T5Tokenizer.from_pretrained(str(OUTPUT_BASE / 'final'))
    smoke_model = T5ForConditionalGeneration.from_pretrained(str(OUTPUT_BASE / 'final')).to(device)
    sample = random.choice(list(zip(val_inputs, val_targets)))
    print('\n[SmokeTest] Input snippet:')
    print(sample[0][:250])
    enc = smoke_tokenizer(sample[0], return_tensors='pt', truncation=True, padding='max_length', max_length=MAX_SOURCE_LEN).to(device)
    out_ids = smoke_model.generate(enc['input_ids'], max_length=256, num_beams=4)
    pred = smoke_tokenizer.decode(out_ids[0], skip_special_tokens=True)
    print('[SmokeTest] Target:', sample[1][:250])
    print('[SmokeTest] Pred  :', pred[:250])
except Exception as e:
    print('[SmokeTest] Failed:', e)

print('\n[Done] Unified single-cell pipeline complete.')

In [ ]:
# Debug Cell: Diagnose out-of-range token IDs during evaluation
# Run AFTER the unified training cell fails, without re-initializing kernel, to inspect offending IDs.
import torch, itertools, json
from collections import Counter

print('[Debug] Starting OOB token diagnostics...')

# 1. Basic tokenizer / sentencepiece stats
try:
    piece_size = tokenizer.sp_model.get_piece_size()
except Exception:
    piece_size = tokenizer.sp_model.piece_size() if hasattr(tokenizer.sp_model,'piece_size') else None
print(f"[Debug] SentencePiece piece_size: {piece_size}")
print(f"[Debug] len(tokenizer): {len(tokenizer)}")

# 2. Inspect embedding matrix size
try:
    embed_rows = model.get_input_embeddings().weight.shape[0]
    print(f"[Debug] Embedding rows: {embed_rows}")
except Exception as e:
    print(f"[Debug] Could not read embedding size: {e}")

# 3. Force a single eval batch forward + generate to capture raw ids
sample_inputs = val_inputs[:4]
enc = tokenizer(sample_inputs, return_tensors='pt', truncation=True, padding='max_length', max_length=128).to(model.device)
with torch.no_grad():
    gen = model.generate(enc['input_ids'], max_length=64, num_beams=1, do_sample=False)
print(f"[Debug] Generated shape: {gen.shape}")

# 4. Collect all ids
all_ids = gen.view(-1).tolist()
min_id, max_id = min(all_ids), max(all_ids)
print(f"[Debug] Min id: {min_id} | Max id: {max_id}")

# 5. Find offending ids
bad_ids = sorted({tid for tid in all_ids if tid < 0 or (piece_size is not None and tid >= piece_size)})
print(f"[Debug] Offending ids count: {len(bad_ids)}")
if bad_ids:
    print(f"[Debug] First up to 20 offending ids: {bad_ids[:20]}")
    freq = Counter([tid for tid in all_ids if tid in bad_ids])
    print('[Debug] Top offending id frequencies:', freq.most_common(10))
else:
    print('[Debug] No offending ids in test generation batch.')

# 6. Map a few good/bad tokens back to pieces safely
safe_sample_ids = [i for i in all_ids if 0 <= i < (piece_size or len(tokenizer))][:10]
print('[Debug] Sample valid tokens -> pieces:')
for i in safe_sample_ids:
    try:
        print(i, '->', tokenizer.sp_model.IdToPiece(i))
    except Exception as e:
        print(i, '-> <error>', e)

print('[Debug] Diagnostics complete. If bad_ids non-empty, we will patch safe_compute next to hard clamp to piece_size-1.')

In [ ]:
# Patch Cell: Harden safe_compute with piece_size clamp and fallback manual decoding
from typing import Iterable
import torch

print('[Patch] Installing hardened safe_compute with piece_size clamp...')

piece_size = None
try:
    piece_size = tokenizer.sp_model.get_piece_size()
except Exception:
    if hasattr(tokenizer.sp_model, 'piece_size'):
        piece_size = tokenizer.sp_model.piece_size()

if piece_size is None:
    print('[Patch] WARNING: Could not obtain piece_size; will use len(tokenizer).')
    piece_size = len(tokenizer)

pad_id = tokenizer.pad_token_id or 0

# Manual decode to avoid sentencepiece exceptions mid-loop

def manual_decode(batch_ids: Iterable[Iterable[int]]):
    texts = []
    for seq in batch_ids:
        pieces = []
        for tid in seq:
            if tid == pad_id:
                continue
            if 0 <= tid < piece_size:
                try:
                    pieces.append(tokenizer.sp_model.IdToPiece(int(tid)))
                except Exception:
                    # skip any anomalous id
                    continue
        # Basic post-processing similar to T5 decode cleanup
        text = ''.join(pieces).replace('▁', ' ').strip()
        texts.append(text)
    return texts

# Re-wrap compute
old_base_compute = base_compute if 'base_compute' in globals() else None

if old_base_compute is None:
    print('[Patch] ERROR: base_compute not found; cannot wrap. Run unified cell first.')
else:
    def hardened_safe_compute(eval_pred):
        preds, labels = eval_pred
        if isinstance(preds, tuple):
            preds = preds[0]
        tensor_mode = isinstance(preds, torch.Tensor)
        if tensor_mode:
            seqs = preds.tolist()
        else:
            seqs = preds
        # Clamp
        oob = 0
        for s in seqs:
            for i, tid in enumerate(s):
                if tid < 0 or tid >= piece_size:
                    s[i] = pad_id
                    oob += 1
        if oob:
            print(f"[Harden] Clamped {oob} OOB ids to pad_id {pad_id}")
        # Try normal decode; if it fails, fallback to manual
        try:
            if tensor_mode:
                preds_tensor = torch.tensor(seqs, device=preds.device)
            else:
                preds_tensor = torch.tensor(seqs)
            return old_base_compute((preds_tensor, labels))
        except Exception as e:
            print('[Harden] Standard decode failed, using manual fallback:', e)
            # Manual decode path replicating base_compute metric usage
            manual_texts = manual_decode(seqs)
            # Decode labels normally with masking
            label_lists = []
            for label_seq in labels:
                label_lists.append([(l if l != -100 else pad_id) for l in label_seq])
            decoded_labels = tokenizer.batch_decode(label_lists, skip_special_tokens=True)
            # Metrics direct
            rouge_res = metric_rouge(manual_texts, decoded_labels)
            bleu_res  = metric_bleu(manual_texts, decoded_labels)
            meteor_res= metric_meteor(manual_texts, decoded_labels)
            bert_res  = metric_bertscore(manual_texts, decoded_labels)
            metrics = {}
            for d in (rouge_res, bleu_res, meteor_res, bert_res):
                metrics.update({k: float(v) for k,v in d.items()})
            return metrics
    compute_metrics = hardened_safe_compute
    trainer.compute_metrics = compute_metrics
    print('[Patch] Hardened safe_compute installed. Re-run evaluation or training resume.')